In [14]:
import fitz  # PyMuPDF
import openai

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text.strip()

In [15]:
import openai

In [16]:
import fitz

In [17]:
import os

In [18]:
os.getcwd()

'C:\\Users\\lawre\\OneDrive\\Documents\\Shiny\\first shiny app'

In [19]:
fitz.open("VAP66848.pdf")

Document('VAP66848.pdf')

In [20]:
pdf_doc = fitz.open("LAP62906.pdf")

In [21]:
pdf_doc.page_count

143

In [22]:
metadata = pdf_doc.metadata

In [23]:
metadata

{'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'iText® 5.5.1 ©2000-2014 iText Group NV (National Association of Insurance Commissioners; licensed version); modified using iTextSharp™ 5.5.5 ©2000-2014 iText Group NV (AGPL-version)',
 'creationDate': "D:20241231054442-06'00'",
 'modDate': "D:20241231070127-05'00'",
 'trapped': '',
 'encryption': None}

In [24]:
metadata = pdf_doc.metadata
print("Metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

Metadata:
  format: PDF 1.4
  title: 
  author: 
  subject: 
  keywords: 
  creator: 
  producer: iText® 5.5.1 ©2000-2014 iText Group NV (National Association of Insurance Commissioners; licensed version); modified using iTextSharp™ 5.5.5 ©2000-2014 iText Group NV (AGPL-version)
  creationDate: D:20241231054442-06'00'
  modDate: D:20241231070127-05'00'
  trapped: 
  encryption: None


In [25]:
# Get Table of Contents
toc = pdf_doc.get_toc(simple=True)  # simple=True returns list of lists with [level, title, page number]

print("Table of Contents:")
for entry in toc:
    level, title, page_num = entry
    indent = "  " * (level - 1)
    print(f"{indent}- {title} (Page {page_num})")

Table of Contents:
- AMSI-134239578 (Page 1)
  - Table of Contents (Page 1)
  - Filing at a Glance (Page 4)
    - General Information (Page 5)
    - Filing Description (Page 5)
    - Company and Contact (Page 5)
    - Filing Fees (Page 6)
  - Correspondence Summary (Page 7)
  - Disposition (Page 8)
  - Objection Letter (Page 10)
  - Objection Letter (Page 11)
  - Response Letter (Page 13)
  - Response Letter (Page 14)
  - Note To Filer (Page 19)
  - Post Submission Update Request Processed On 09/17/2024 (Page 20)
  - Rate Information (Page 21)
  - Rate/Rule Schedule (Page 22)
  - Supporting Document Schedules (Page 28)
  - Superseded Schedule Items (Page 143)


In [26]:
# Iterate through pages to get their sizes
for page_num in range(pdf_doc.page_count):
    page = pdf_doc[page_num]
    rect = page.rect  # Rectangle object
    width = rect.width
    height = rect.height
    print(f"Page {page_num + 1}: Width = {width}, Height = {height}")

Page 1: Width = 612.0, Height = 792.0
Page 2: Width = 612.0, Height = 792.0
Page 3: Width = 612.0, Height = 792.0
Page 4: Width = 612.0, Height = 792.0
Page 5: Width = 612.0, Height = 792.0
Page 6: Width = 612.0, Height = 792.0
Page 7: Width = 792.0, Height = 612.0
Page 8: Width = 792.0, Height = 612.0
Page 9: Width = 792.0, Height = 612.0
Page 10: Width = 612.0, Height = 792.0
Page 11: Width = 612.0, Height = 792.0
Page 12: Width = 612.0, Height = 792.0
Page 13: Width = 612.0, Height = 792.0
Page 14: Width = 792.0, Height = 612.0
Page 15: Width = 792.0, Height = 612.0
Page 16: Width = 792.0, Height = 612.0
Page 17: Width = 792.0, Height = 612.0
Page 18: Width = 792.0, Height = 612.0
Page 19: Width = 612.0, Height = 792.0
Page 20: Width = 612.0, Height = 792.0
Page 21: Width = 792.0, Height = 612.0
Page 22: Width = 792.0, Height = 612.0
Page 23: Width = 792.0, Height = 612.0
Page 24: Width = 612.0, Height = 792.0
Page 25: Width = 792.0, Height = 612.0
Page 26: Width = 612.0, Height = 7

In [27]:
# Extract text from all pages
for page_num in range(pdf_doc.page_count):
    page = pdf_doc[page_num]
    text = page.get_text()
    print(f"--- Page {page_num + 1} ---")
    print(text)

--- Page 1 ---
Table of Contents
 
User Usage Agreement
Attachments
Usage Agreement
Usage Agreement.pdf
Rate-Rule Attachments
(ex. Document Name      Attachment Name)
HOH Rate Pages
HO3_202411 Clean Rates.pdf
HOS Rate Pages
LA HO5 202411 Rate Factor Pages.pdf
HOH ROC
Exhibit HO3_202411 ROC.pdf
HOS ROC
LA HO5 202411 ROC.pdf
Supporting Document
Attachments
(ex. Supporting Document Name      Attachment Name)
Explanatory Memorandum (Rates
and Rules)
Explanatory Memo - LA.pdf
      Explanatory Memorandum (Rates
and Rules)
Explanatory Memo - LA (1).pdf
Justification (Rates and Rules)
LA HO3 202411 Actuarial Summary.pdf
Justification (Rates and Rules)
LA HO5 202411 Actuarial Summary.pdf
Last Action Letter
Last Action Letter (HO3).pdf
Last Action Letter
Last Action Letter (HO5).pdf
Manual Pages (Rates and Rules)
HO3_202411 Side by Side Markup.pdf
Manual Pages (Rates and Rules)
HO5_202411 Side by Side Markup.pdf
Rate Revisions Exhibits for Home
Owners
Exhibit H HO3_202411.pdf
Rate Revisions Exh

In [36]:
import os
import time
import shutil
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# --- Configuration ---
# Path to your Downloads folder
DOWNLOADS_FOLDER = os.path.expanduser("~/Downloads")
# Path to the working directory for your pipeline
WORKING_DIRECTORY = os.path.expanduser("~/Documents/PDF_Working_Directory")
# File extension to monitor (case insensitive)
FILE_EXTENSION = ".pdf"

# Create the working directory if it doesn't already exist
if not os.path.exists(WORKING_DIRECTORY):
    os.makedirs(WORKING_DIRECTORY)
    print(f"Created working directory: {WORKING_DIRECTORY}")

class PDFHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Only consider files (not directories) that end with .pdf
        if not event.is_directory and event.src_path.lower().endswith(FILE_EXTENSION):
            print(f"Detected new file: {event.src_path}")
            self.move_file(event.src_path)

    def on_moved(self, event):
    # This event fires when a file is renamed or moved.
        if not event.is_directory and event.dest_path.lower().endswith(FILE_EXTENSION):
            print(f"Detected moved/renamed file: {event.dest_path}")
            self.move_file(event.dest_path)

    def move_file(self, src_path):
        filename = os.path.basename(src_path)
        destination = os.path.join(WORKING_DIRECTORY, filename)
        
        base, ext = os.path.splitext(filename)
        counter = 1
        while os.path.exists(destination):
            destination = os.path.join(WORKING_DIRECTORY, f"{base}_{counter}{ext}")
            counter += 1

        while not self.is_file_ready(src_path):
            time.sleep(1)
        
        try:
            shutil.move(src_path, destination)
            print(f"Moved '{filename}' to '{destination}'")
        except Exception as e:
            print(f"Error moving file '{filename}': {e}")

    def is_file_ready(self, file_path):
        """
        Check if a file is ready by attempting to open it.
        If it’s still being written (or locked), an IOError may occur.
        """
        try:
            with open(file_path, 'rb'):
                return True
        except IOError:
            return False

def main():
    event_handler = PDFHandler()
    observer = Observer()
    observer.schedule(event_handler, path=DOWNLOADS_FOLDER, recursive=False)
    observer.start()
    print(f"Monitoring '{DOWNLOADS_FOLDER}' for new PDF files...")
    
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
        print("Stopping the PDF mover.")
    observer.join()

#if __name__ == "__main__":
#    main()

In [37]:
main()

Monitoring 'C:\Users\lawre/Downloads' for new PDF files...
Detected moved/renamed file: C:\Users\lawre/Downloads\NVP63436 (1).pdf
Moved 'NVP63436 (1).pdf' to 'C:\Users\lawre/Documents/PDF_Working_Directory\NVP63436 (1).pdf'
Detected moved/renamed file: C:\Users\lawre/Downloads\NVP63436 (1).pdf
Moved 'NVP63436 (1).pdf' to 'C:\Users\lawre/Documents/PDF_Working_Directory\NVP63436 (1)_1.pdf'
Detected moved/renamed file: C:\Users\lawre/Downloads\NVP63436 (1).pdf
Moved 'NVP63436 (1).pdf' to 'C:\Users\lawre/Documents/PDF_Working_Directory\NVP63436 (1)_2.pdf'
Detected moved/renamed file: C:\Users\lawre/Downloads\NVP63436 (1).pdf
Moved 'NVP63436 (1).pdf' to 'C:\Users\lawre/Documents/PDF_Working_Directory\NVP63436 (1)_3.pdf'
Detected moved/renamed file: C:\Users\lawre/Downloads\NVP63436 (1).pdf
Moved 'NVP63436 (1).pdf' to 'C:\Users\lawre/Documents/PDF_Working_Directory\NVP63436 (1)_4.pdf'
Detected moved/renamed file: C:\Users\lawre/Downloads\SDP59285.pdf
Moved 'SDP59285.pdf' to 'C:\Users\lawre/D

In [29]:
os.path.expanduser("~/Downloads")

'C:\\Users\\lawre/Downloads'

In [30]:
os.getcwd()

'C:\\Users\\lawre\\OneDrive\\Documents\\Shiny\\first shiny app'